
# Agent记不住历史信息？Memory Transformer实现长期记忆建模

## 1. 问题：Agent 为什么记不住历史？

大多数 AI Agent 依赖固定长度的上下文窗口（如 8 条消息）。当对话轮次超过限制，早期信息被丢弃。

典型问题：
- 用户第1轮强调“关注排他性条款”
- 第10轮提问“总结风险点”时，该指令已被清除
- Agent 失去目标一致性，决策偏离初衷

RAG 虽能检索历史，但无法判断“什么重要”，也无法动态管理记忆。



## 2. 核心思想

Memory Transformer 通过三个机制解决记忆问题：

### （1）分层记忆
- **短期记忆**：存放近期对话，用于上下文拼接
- **长期记忆**：存储高价值信息，支持跨轮次保留

### （2）重要性感知
自动评估每条信息的重要性：
```python
def _calculate_importance(self, content: str) -> float:
    score = 0.3
    keywords = ['排他性', '违约金', '重要', '终止']
    for kw in keywords:
        if kw in content:
            score += 0.15
    return min(1.0, score)
```
高分信息进入长期记忆，低分仅存短期。



## 3. 系统架构

- `MemoryVector`：封装内容、重要性、嵌入向量、访问次数
- `MemoryStorage`：管理分层存储与索引
- `MemoryRetriever`：执行跨层级语义检索
- `MemoryTransformer`：主控流程



## 4. 实战演示

### 场景：10轮合同审查
```
1. 用户: 特别关注排他性条款
...
10. 用户: 总结排他性条款的风险
```

| 方案 | 是否保留初始指令 |
|------|------------------|
| 传统上下文（6条） | 否（被挤出） |
| Memory Transformer | 是（高重要性 → 长期记忆） |

系统成功召回所有相关记忆，生成完整摘要。

## 5. 代码实现

## 6. 与 RAG 的区别

| 维度 | RAG | Memory Transformer |
|------|-----|---------------------|
| 信息筛选 | 无 | 重要性评分驱动 |
| 存储结构 | 扁平 | 分层（短期+长期） |
| 写入控制 | 全量 | 智能调度 |
| 目标一致性 | 弱 | 强 |

**本质不同**：  
RAG 是“图书馆检索”，Memory Transformer 是“有经验的专家”。



## 7. 结语

Memory Transformer 不是更大的上下文，而是一种**可持续的记忆系统**。

它让 AI Agent：
- 知道什么重要
- 记住关键约束
- 在复杂任务中保持连贯性

未来，记忆不应是附加功能，而应是 Agent 的核心组件。
